In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Train/Sylhet Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Test/Sylhet Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Validation/Sylhet Validation Translation.csv")

In [4]:
train_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,কেমন আছো ?,kemon acho?,ভালা আছনি?,vala achoni?,Sylhet,How are you?
1,আজকে আমার মন ভালো নেই,ajke amr mon valo nei,আইজকু আমার মন ভালা নায়,aijku amar mon vala nay,Sylhet,I'm not feeling well today
2,তুমি কি করো ?,tumi ki koro?,তুমি কিতা খরো?,tumi kita khoro?,Sylhet,what are you doing
3,এই গরমে আমার কিছু ভালো লাগে না,ei gorome amar kichu valo lage na,অউ গরমো আমার কুনতা ভালা লাগের না,ou gormo amar kunta vala lager na,Sylhet,I don't like anything this summer
4,ছেলেটি সাদা রঙয়ের একটি শার্ট পরে এসেছিল,cheleti sada ronger ekti shirt pore eshechilo,ফুয়াটায় এখটা সাদা রংগর শার্ট পিন্দিয়া আইছিল,fuyatay ekhta sada ronggor shirt pindiya aichilo,Sylhet,The boy came wearing a white shirt


In [5]:
train_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
1870,কাউকে মিথ্যা বলে খুশী করার থেকে সত্যি বলে কাঁদ...,kauke mittha bole khushi korar theke sotti bol...,কাউরে মিচা বলিয়া খুশির করার চেয়ে হাচা বলে কানদ...,kaure mica boliya khushir korar cheye hacha bo...,Sylhet,It's better to be happy for telling a lie than...
1871,"একে অপরের প্রতি যতো বেশী বিশ্বাস থাকবে, তাদের ...","eke oporer proti joto beshi bisshas thakbe, ta...","একে অপরের পতি যতো বেশি বিশ্বাস থাকবো, তাদের ভ...","eke oprer poti joto beshi bisshas thakbo, tad...",Sylhet,"The more trust they have in each other, the he..."
1872,বৃষ্টি বেড়ে তাপমাত্রা কমার সুখবর দিয়েছে আবহা...,brishti bere tapmatra komar sukhobor diyeche a...,মেগ বাড়িয়া তাপমাত্রা কমার সুখবর দিয়েছে আবহাওয়া...,meg bariya tapmatra komar sukhobor diyeche abh...,Sylhet,Meteorological office has given the good news ...
1873,পুরো বর্ষা মৌসুমে হাতে গোনা কয়েকদিন মাত্র বৃষ...,puro borsha moushume hate gona koyekdin matro ...,ফুরো মেগ মৌসুমো হতে গনা কয়েকদিন মাত্র মেগ অওয়া...,furo meg mousumo hote gona koyekdin matro meg ...,Sylhet,After only a handful of days of rain in the en...
1874,খাওয়ার অনেক আগেই যদি ভাত রান্না শেষ হয়ে গিয়ে থ...,khaoyar onek agei jodi vat ranna shesh hoye gi...,খাওয়ার অনেক আগেই যদি ভাত রান্দা শেষ হয়ে অইয়া গ...,khaoyar onek agei jodi vat randa shesh hoye oi...,Sylhet,If the rice is done cooking long before eating...


In [6]:
test_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,তোমার আব্বু কেমন আছে?,tomar abbu kemon ache?,তোমার আব্বা বালা আছইন নি?,tomar abba bala achoin ni?,Sylhet,How is your father?
1,আমার বড় বোনের আজকে মন ভালো নেই,amar boro boner ajke mon valo nei,আমার বড় বইনর আইজ মন ভালা নায়,amar boro boinor aij mon vala nay,Sylhet,My elder sister is not feeling well today
2,তুমি কি আমাকে এই কাজটি করে দিতে পারবে?,tumi ki amake ei kajti kore dite parbe?,তুমি কিতা মোরে এই কাজটা করাই দিতা পারবা নি?,tumi kita more ei kajta korai dita parba ni?,Sylhet,Can you do this for me?
3,এই গরমে বাহিরে বের হতে ইচ্ছা করে না,ei gorome bahire ber hote iccha kore na,এই গরমো বাইরে বাইর অইতে জানে কয় না,ei gormo baire bair oite jane koy na,Sylhet,Don't want to go out in this summer
4,তোমার সব কথাই ভুল,tomar sob kothai vul,তোমার সব মাত ভুল,tomar sob mat vul,Sylhet,Everything you say is wrong


In [7]:
test_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
370,তুমি কি আমাকে এক গ্লাস পানি এনে দিতে পারবে ?,tumi ki amake ek glass pani ene dite parbe ?,তুমি কি আমারে এক গ্লাস ফানি এনে দিতায় ফারবা?,tumi ki amare ek glass fani ene ditay farba?,Sylhet,Can you get me a glass of water?
371,আমি পারবো না পানি দিতে,ami parbo na pani dite,আমি পারবো না ফানি দিতে,ami parbo na fani dite,Sylhet,I can't give water
372,তোমার ভাই পড়ালেখাতে অনেক মেধাবি,tomar vai poralekhate onek medhabi,তোমার ভাই ফড়ালেখাতে বহুত মেধাবি,tomar vai foralekhate bohut medhabi,Sylhet,Your brother is very talented in studies
373,আচ্ছা বলো দেখি বাংলাদেশে কয়টি জেলা?,accha bolo dekhi bangladeshe koyti jela?,আইচ্চা কও দেখি বাঙলাদেশো কয়টা জেলা?,aiccha kow dekhi bangladesho koyta jela?,Sylhet,Well tell me how many districts in Bangladesh?
374,সামনের দিকে যেয়ে মেয়েটি অনেক হাসবে,samner dike jeye meyeti onek hashbe,সামনর দিকে যাইয়া ফুড়িটা বহুত হাসবো,samnor dike jaiya furita bohut hashbo,Sylhet,The girl will smile a lot going forward


In [8]:
validation_data.head(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
0,বাংলাদেশে ৬৪ টা জেলা,bangladeshe ৬৪ ta jela,বাংলাদেশো ৬৪ টা জেলা,bangladesho ৬৪ ta jela,Sylhet,64 districts in Bangladesh
1,আমরা সবাই গতকাল বাহিরে গিয়েছিলাম,amra sobai gotokal bahire giyechilam,আমরা হকল গতকাইল বাহিরো গিয়েছিলাম,amra hokol gotokail bahiro giyechilam,Sylhet,We all went out yesterday
2,তোমার কথা বলার ধরন অনেক সুন্দর,tomar kotha bolar dhoron onek sundor,তোমার মাত বলার ধরন বহুত সুন্দর,tomar mat bolar dhoron bohut shundor,Sylhet,Your way of speaking is very nice
3,বরিশালের মানুষ কেমন হয়?,barishaler manush kemon hoy?,বরিশালর মানুষ কিলা অইন?,barishalor manush kila oin?,Sylhet,How are the people of Barisal?
4,খুলনা জেলা কি অনেক সুন্দর?,khulna jela ki onek sundor?,খুলনা জেলা কিতা বহুত সুন্দর?,khulna jela kita bohut shundor?,Sylhet,Khulna district is very beautiful?


In [9]:
validation_data.tail(5)

,bangla_speech,banglish_speech,sylhet_bangla_speech,sylhet_banglish_speech,region_name,english_speech
245,তুমি যাও এখান থেকে,tumi jao ekhan theke,তুমিতাইন হরো ওখান তাকি,tumitain horo okhan taki,Sylhet,you go from here
246,আপনি যান এখান থেকে,apni jan ekhan theke,আফনে হরোউকা ওখান তাকি,afne horouka okhan taki,Sylhet,You go from here
247,তুই যা এখান থেকে,tui ja ekhan theke,তুই হর ওখান তাকি,tui hor okhan taki,Sylhet,you go from here
248,আপনারা পার্কে ঘুরে বেড়ান,apnara parke ghure beran,আফনারা পার্ক-ও ঘুরিয়া বেড়াউকা,afnara park-o ghuriya berauka,Sylhet,You go for a walk in the park
249,উনি চেষ্টা করেন,uni cheshta koren,তাইন চেষ্টা করোইন,tain cheshta koroin,Sylhet,He tries


In [10]:
!pip install transformers torch pandas

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [11]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.9 MB/s eta 0:00:00


In [12]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=667ff63cce9b2decacf871f9aa2620b9411db6a1eb81219dbf30ef61674fb2c5
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
!pip install sentencepiece

In [14]:
!pip install transformers[sentencepiece]

In [15]:
!transformers-cli cache clear

2024-06-10 08:55:20.824622: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 08:55:20.824673: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 08:55:20.880807: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-10 08:55:20.889129: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-10 08:55:21.929819: W tensorflow/comp

In [16]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.1 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

In [ ]:
!pip install --upgrade pip

In [ ]:
#!pip install transformers==4.10.3

In [ ]:
!pip install torch transformers

In [ ]:
#!pip install accelerate==0.20.3

In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Train/Sylhet Train Translation.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Test/Sylhet Test Translation.csv")
validation_data = pd.read_csv("/content/drive/MyDrive/Vashantor_CSV_Format/Validation/Sylhet Validation Translation.csv")

In [ ]:
train_data.head()

In [ ]:
train_data = train_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
train_data.head()

In [ ]:
test_data = test_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
test_data.head()

In [ ]:
train_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
train_data.head()

In [ ]:
# Rename the columns to match the expected format
test_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
test_data.head()

In [ ]:
validation_data.head()

In [ ]:
validation_data = validation_data.drop(columns=['banglish_speech ','sylhet_banglish_speech ','region_name ','english_speech'],axis=1)
validation_data.head()

In [ ]:
# Rename the columns to match the expected format
validation_data.rename(columns={'bangla_speech ': 'input_text', 'sylhet_bangla_speech '	: 'labels'}, inplace=True)
validation_data.head()

In [ ]:
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer
#https://huggingface.co/docs/transformers/model_doc/mt5
model_name = "google/mt5-small" # The variations it has -> mt5-small: 6, mt5-base: 12,mt5-large: 24, mt5-xl: 24, mt5-xxl: 24
model = MT5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
'''from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_tokenizer.json")

In [ ]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [ ]:
""""model_name = "csebuetnlp/banglat5"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
'''
# Save the model
model.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'''

In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [ ]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Mymensing_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Mymensing_tokenizer.json")


In [ ]:
#pip install transformers==4.30


In [ ]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)

#if you have 1,000 training examples and a batch size of 100, you would have 10 iterations in each epoch (1,000 / 100 = 10)
'''
This parameter determines how many small batches are accumulated before performing a weight update.
In your code, it's set to 8, which means you'll accumulate gradients over 8 small batches before performing a weight update.
This effectively simulates a larger batch size without requiring more GPU memory.
So, you are updating the model's weights less frequently compared to the number of actual batches processed.
'''



'''
Learning rate determines how quickly the model learns from the data.
The learning rate scheduler type is set to "cosine_with_restarts," which is a type of learning rate schedule.
 Warmup steps are the number of initial training steps with a smaller learning rate, and weight decay introduces L2 regularization to the optimizer.
'''



'''
Number of Iterations per Epoch = Number of Training Samples / Batch Size
Total Iterations = Number of Iterations per Epoch * Number of Epochs
'''
# Define the TrainingArguments
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Movies',
    num_train_epochs=30,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Align save strategy with evaluation strategy
    ##save_total_limit=1,
    #save_steps=700,
    learning_rate=1e-3,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
    #load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/Movies',
    logging_steps=10,  # Log every 10 steps,
)


In [ ]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [ ]:

# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [ ]:
# Fine-tune the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2717: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,0.020500,0.082099
1,0.025100,0.074538
2,0.019200,0.074786
4,0.014400,0.080060
5,0.017700,0.075762
6,0.014100,0.078575
8,0.010800,0.080162
9,0.009200,0.082601
10,0.007300,0.085556
12,0.004100,0.093614


# **Saving model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# Save the model
model.save_pretrained("/content/drive/MyDrive/Movies/Mymensing_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Movies/Mymensing_tokenizer.json")


('/content/drive/MyDrive/Movies/Mymensing_tokenizer.json/tokenizer_config.json',
 '/content/drive/MyDrive/Movies/Mymensing_tokenizer.json/special_tokens_map.json',
 '/content/drive/MyDrive/Movies/Mymensing_tokenizer.json/spiece.model',
 '/content/drive/MyDrive/Movies/Mymensing_tokenizer.json/added_tokens.json',
 '/content/drive/MyDrive/Movies/Mymensing_tokenizer.json/tokenizer.json')

# **Loading trained model and tokenizer**

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Mymensing_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Mymensing_tokenizer.json")


You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 81.0 MB/s eta 0:00:00


In [ ]:
!pip install jiwer

In [ ]:
# Move the model to the device (CPU or GPU)
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [ ]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.2 MB/s eta 0:00:00


# **Loading evaluation metrics**

In [ ]:
import torch
import Levenshtein
from evaluate import load
# Define the move_to_device function
def move_to_device(batch, device):
    if isinstance(batch, torch.Tensor):
        return batch.to(device)
    elif isinstance(batch, list):
        return [move_to_device(item, device) for item in batch]
    elif isinstance(batch, dict):
        return {key: move_to_device(value, device) for key, value in batch.items()}
    else:
        return batch  # If it's not a tensor, list, or dict, leave it as is

# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)
cer_metric = load("cer")
wer_metric = load("wer")
meteor = load('meteor')
exact_match_metric = load("exact_match")

# Load BLEU and ROUGE metrics
bleu_metric = load("bleu")
rouge_metric = load('rouge')

# Initialize lists to store generated translations and references
generated_translations = []
references = []

# Generate translations for the test dataset
for batch in test_dataloader:
    # Move the batch to CUDA
    batch = move_to_device(batch, 'cuda')

    input_text = batch['input_ids']  # Access the input_text using the correct key
    labels = batch['labels']  # Access the labels using the correct key

    # Generate translations
    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)

    # Move the translation_ids to CPU to decode
    translation_ids = translation_ids.to('cpu')

    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)

    generated_translations.extend(generated_translation)
    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs

# Make sure to move generated_translations back to CPU for evaluation if necessary
generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]



In [ ]:
print("Number of generated translations:", len(generated_translations))
print("Number of references:", len(references))

Number of generated translations: 375
Number of references: 375


In [ ]:
print(generated_translations)

['তোমার তোমারে তোমার আমি হের রাই', 'আমি হের আমি থাইক্কা', 'আমার ছেড়ি লাইজ্ঞা ঠে', 'তোমার আমি তোমার হেরা গেসি', 'আমি তোমার কথা কইরা কইরা কথা কথা কথা কইরা কথা কইরা কইসি', 'তোমার আমি তোমারে তোইক্কা থাইক্কা কথা কইরা হইব', 'ছেড়া তোমার আমি তোমারে আমি আসিল', 'আমি তোমার ছেড়া তোমার ছেড়া তোমার', 'আফনের জামাইতো আমার আমি আমার আমি আমি আমি আসিল', 'আমি একটা তোমার ঠিক না', 'ছেড়া ছেড়া ছেড়া কথা কইরা থা তোমার থাইক্কা', 'তোমার আমি তো তোমারে বিদেশ করতাম', 'আফনের লাইজ্ঞা লাইজ্ঞা মেলা', 'আমার পরিবারে মধ্যে পড়ার নাই', 'আফনে মেলা মেলা কথা কথা কই কইরা আসিল', 'আমার জামাই কথা কথা কথা কথা কথা কথা কইসে', 'আমি তোমার ছেড়া ভাইয়ের কথা কথা কথাইক্কা নাই', 'আমি তোমারে আমি থা পারতা', 'আমার আব্বা জামাইস্কুলে', 'আমি তোমার তোমার কিন্তু হের হের', 'তোমারে থাইক্কা আফনের লাইজ্ঞা ভালবাসে', 'আমি একটা কথা কইরা লাইজ্ঞা', 'তোমার আমি তোমার কাউরে?', 'আমার তোমার লাইজ্ঞা কথা কথা কথা কথা কথা কথা কথা কথা কইসে', 'আমি তোমারে এইডা কথা কইরা ঠিক', 'আমারে মেলা তোমার থা আসি', 'আমার জামাই আমার মার জামাইরা ছেড়ি ভালো', 'আমি ফুটবল কথা থাইক্

In [ ]:
print(references)

['তোমার বাপ কেমত আসে?', 'আমার বড় বইনের আইজ মন ভালা নাই', 'তুমি কিতা আমারে এই কামডা কইরা দিতা ফারবা?', 'এই গরমে বাইরে বাইর হইতে ইচ্ছা করে না', 'তোমার বেগ কথাই ভুল', 'তোমার মত খারাপ ছেড়া আমি আর একটাও দেখসিনা', 'বিয়ার লাইজ্ঞা আম্মা আমার লাইজ্ঞা ছেড়া খুজতাসে', 'আব্বা আমারে কালকে একটা ছেড়ার সাথে দেখা করতে কইসিন', 'ছেড়াডারে আমার এক্কেরেই ভালা লাগসেনা', 'ছেড়িডার গোলাপি রংগের শাড়ি পইরা আইসিল', 'আমার দুইদিন বাদে বিয়া হইব', 'ছেড়াডার এহন আর বিকালে খেলত বাইর হয় না', 'ছেড়িডা সকালে ঘুম থাইক্কা উইত্থাই পরতে বয়', 'আমার চাচা বিদেশ থাইক্কা কালকে বাংলাদেশে আইসে', 'আমার বাসার দারওয়ান অনেক ভালা মানুষ', 'আমার বন্ধু খুলনা ঘুরতে গিয়েছে', 'আমার ঘুরতে মেলা ভালা লাগে', 'আমি বিদেশে যায়া মেলা ঘুরতাম চাই', 'তুমি আমারে বিরক্ত কইরো না', 'হেয় আমারে মেলা অপমান করসে', 'আমার সকালে উইত্থা বই পরতে মেলা ভালা লাগে', 'আমার ছোট ভাই এক্কেরেই পড়ালেহা করতে চায় না', 'আমার ছোট ভাই ইস্কুলে যাইতে চায় না', 'আমার ছোট ভাইয়ের বিদ্যালয়ে গিয়ে পড়ালেখা করতে ভাল লাগে না', 'আমি একদিন ইস্কুলের মাঠে মাথা ঘুরায়া পইরা গেসি

In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

In [ ]:
total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)


Accuracy: 0.6133333333333333


# **Saving input text(Mymensing Language) , labels(normal bangla) and generated translations to csv**

In [ ]:
import pandas as pd

# Create a DataFrame to store translations
translation_df = pd.DataFrame({
    'input_text': test_data['input_text'],  # Assuming 'test_data' contains your test dataset
    'labels': references,
    'translations': generated_translations
})

# Save translations to a CSV file
# Save translations to a CSV file
translation_df.to_excel('/content/drive/MyDrive/Dataset/Mymensing_translations_mT5.xlsx', index=False)
